<a href="https://colab.research.google.com/github/patelharsh94/HP-Nano-GPT/blob/main/HP_Nano_GPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

SyntaxError: invalid syntax (<ipython-input-1-3d6cfb4a5e4d>, line 1)